# <B>MOD 4 PROJECT

<b>main factors in life satisfaction:</b>
   <br>health status
   <br>mental health
   <br>income
   <br>employment status

# <b>DATA CLEANING

<B>IMPORTING DATA:

In [ ]:
#importing necessary packages
import numpy as np
from scipy import stats 
import math
import statsmodels.api as sm 
import pylab as py
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from itertools import combinations, product
from sklearn.model_selection import KFold,cross_val_predict,cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression,RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols

<B>LOOKING THROUGH THE DATASET:

In [ ]:
#importing pandas and the csv files
import pandas as pd
df = pd.read_csv('analytic_data2019.csv')
df.head()

In [ ]:
#checking shape
df.shape

In [ ]:
#display list of column values
list(df.columns.values)

In [ ]:
#checking for nan values
df.isna().sum()

In [ ]:
#checking for null values
df.isnull().sum()

In [ ]:
#renaming columns by placing an underscore between words to make it more usable
columns_name = df.columns
new_column_names = []
for column_name in columns_name:
    new_column_names.append(column_name.replace(" ", "_"))
print(new_column_names[:10])
df.columns = new_column_names

In [ ]:
df = df.iloc[2:]

In [ ]:
df = df.loc[((df['County_FIPS_Code']!='000') & (df['County_FIPS_Code']!=0))]

In [ ]:
df.head()

In [ ]:
xls = pd.ExcelFile("2019 County Health Rankings Data - v2.xls")
health_rankings = xls.parse(index_col=None, na_values=['NA'])
df.to_csv('file.csv')
health_rankings.head()

In [ ]:
xls = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')
sheet0 = xls.parse(0,skiprows=1) 
sheet1 = xls.parse(1,skiprows=1) 
sheet2 = xls.parse(2,skiprows=1) 
sheet3 = xls.parse(3,skiprows=1) 
sheet4 = xls.parse(4,skiprows=1) 
sheet5 = xls.parse(5,skiprows=1) 
sheet6 = xls.parse(6,skiprows=1)

In [ ]:
sheet4

In [ ]:
sheet3.set_index('FIPS', inplace=True)
sheet5.set_index('FIPS', inplace=True)

In [ ]:
#combined = pd.concat([df, sheet5, sheet3], axis=1, sort=False, join='inner')

combined = pd.concat([df], axis=1, sort=False, join='inner')
combined.shape

In [ ]:
df = combined.copy()

In [ ]:
clean_columns = [x for x in df.columns if
                 ("numerator" not in x) and
                 ("denominator" not in x) and
                 ("CI_low" not in x) and
                 ("CI_high" not in x)]

In [ ]:
df = combined[clean_columns]
df.head()

In [ ]:
#checking the data types for each column
for col in df.columns:
    print(df[col].dtypes, col)

In [ ]:
df1=df[['State_FIPS_Code','County_FIPS_Code','5-digit_FIPS_Code','State_Abbreviation','Name','Release_Year','County_Ranked_(Yes=1/No=0)']]

In [ ]:
df2 = df.iloc[:,7:].astype(float)

In [ ]:
dfClean = df1.merge(df2, left_index=True,right_index=True)
dfClean.head()

In [ ]:
#checking the data types for each column
for col in df.columns:
    print(dfClean[col].dtypes, col)

# <b>EXPLORATORY DATA ANALYSIS

# <B>Mental health:

In [ ]:
dfMH=df[['State_Abbreviation','Name','Poor_mental_health_days_raw_value']]
dfMH.head()

In [ ]:
for col in dfMH.columns:
    print(dfMH[col].dtypes, col)

In [ ]:
#changing the type of mental health days raw value from an object to a float
dfMH['Poor_mental_health_days_raw_value']=dfMH['Poor_mental_health_days_raw_value'].astype('float')

In [ ]:
dfMH.groupby(['State_Abbreviation'])['Poor_mental_health_days_raw_value'].mean()

In [ ]:
#Poor mental health days based on state graph
fig,ax = plt.subplots(1,1,figsize=(10,4))

cnt = dfMH.groupby(['State_Abbreviation'])['Poor_mental_health_days_raw_value'].mean().sort_values(ascending=False).to_frame()
sns.barplot(y = cnt['Poor_mental_health_days_raw_value'], x = cnt.index, palette= 'inferno')
ax.set_title('Poor mental health days based on State')
ax.set_ylabel('mental health days')
plt.xticks(rotation= 'vertical') 
ax.set_xlabel('State')


In [ ]:
sm.qqplot(dfMH['Poor_mental_health_days_raw_value'])

In [ ]:
sns.distplot(dfMH['% Unemployed'])

# <b>Physical health:

In [ ]:
dfFH=df[['State_Abbreviation','Name','Poor_physical_health_days_raw_value']]
dfFH.head()

In [ ]:
#changing the type of physical health days raw value from an object to a float
dfFH['Poor_physical_health_days_raw_value']=dfFH['Poor_physical_health_days_raw_value'].astype('float')

In [ ]:
#Poor mental health days based on state graph
fig,ax = plt.subplots(1,1,figsize=(10,4))

cnt = dfFH.groupby(['State_Abbreviation'])['Poor_physical_health_days_raw_value'].mean().sort_values(ascending=False).to_frame()
sns.barplot(y = cnt['Poor_physical_health_days_raw_value'], x = cnt.index, palette= 'inferno')
ax.set_title('Poor physical health days based on State')
ax.set_ylabel('physical health days')
plt.xticks(rotation= 'vertical') 
ax.set_xlabel('State')

# <b>Income:

In [ ]:
df.head()

In [ ]:
dfI=df[['State_Abbreviation','Median_household_income_raw_value']]

In [ ]:
dfI['Median_household_income_raw_value']=dfI['Median_household_income_raw_value'].astype('float')

In [ ]:
#Poor mental health days based on state graph
fig,ax = plt.subplots(1,1,figsize=(10,4))

cnt = dfI.groupby(['State_Abbreviation'])['Median_household_income_raw_value'].mean().sort_values(ascending=False).to_frame()
sns.barplot(y = cnt['Median_household_income_raw_value'], x = cnt.index, palette= 'ocean')
ax.set_title('Median household income by State')
ax.set_ylabel('Median household income')
plt.xticks(rotation= 'vertical') 
ax.set_xlabel('State')

In [ ]:
sm.qqplot(dfI['Median_household_income_raw_value'])

# <b>Employment status:

In [ ]:
dfNew=df[['State_Abbreviation','Name','Unemployment_raw_value']]

In [ ]:
dfNew.head()

In [ ]:
dfNew['Unemployment_raw_value']=dfNew['Unemployment_raw_value'].astype(float)

In [ ]:
#Poor mental health days based on state graph
fig,ax = plt.subplots(1,1,figsize=(10,4))

cnt = dfNew.groupby(['State_Abbreviation'])['Unemployment_raw_value'].mean().sort_values(ascending=False).to_frame()
sns.barplot(y = cnt['Unemployment_raw_value'], x = cnt.index, palette= 'rainbow_r')
ax.set_title('Unemployment by State')
ax.set_ylabel('Percentage of Unemployed')
plt.xticks(rotation= 'vertical') 
ax.set_xlabel('State')

In [ ]:
sm.qqplot(combine['% Unemployed'])

In [ ]:
sns.distplot(combine['% Unemployed'])

In [ ]:
df.head()

In [ ]:
dfClean.shape

In [ ]:
dfClean2=dfClean.iloc[:,40:]

In [ ]:
sns.heatmap(dfClean2.corr(),annot =True)

In [ ]:
msk = np.random.rand(len(dfClean)) < 0.8

train = dfClean[msk]
test = dfClean[~msk]

y_train = train['Life_Expectancy']

X_train = train.drop(['Life_Expectancy'],axis=1)
X_train = X_train.select_dtypes(exclude=['object'])


#X_train, x_test, y_train, y_test = train_test_split(X,y)

In [ ]:
cros_val = KFold(n_splits=10, random_state=1, shuffle=True)
scaler = MinMaxScaler()

Xs = scaler.fit_transform(X_train)

#X_train, X_test, y_train, y_test = train_test_split(X,y)

linreg = LinearRegression()
baseline = np.mean(cross_val_score(linreg,Xs,y_train,scoring='r2',cv=cros_val))
baseline

In [ ]:
md = train.copy()
md =md.drop(['Mentally_Unhealthy_Days','Diabetic','Years_of_Potential_Life_Lost_Rate',
             'Excessive_Drinking', 'Some_College','Children_in_Poverty','Household_Income'],axis=1)